In [25]:
import json
with open("../corpus/debate_scripts.json") as f, open("output.txt", "w") as out_f:
    data = json.load(f)
    for round in data:
        print("title: ", round["source"]["title"], file=out_f)
        for id, speech in enumerate(round["speeches"]):
            print(f"speech id: {id}", file=out_f)
            for id, ADU in enumerate(speech["ADUs"]):
                print(f"{ADU['id']} {ADU['transcript']}", file=out_f)

In [1]:
import json
import re

with open("../corpus/debate_scripts.json") as f:
    data = json.load(f)
    for round in data:
        title = re.sub(r'[\/:*?"<>|]', '', round["source"]["title"])  # Remove invalid filename characters
        with open(f"dummy_timestamps/{title}.txt", "w") as out_f:
            for id, speech in enumerate(round["speeches"]):
                print(f"speech id: {id}", file=out_f)
                for id, ADU in enumerate(speech["ADUs"]):
                    print(f"{ADU['id']} {ADU['transcript']}", file=out_f)

In [25]:
import json
import re

with open("../corpus/debate_scripts.json") as f:
    data = json.load(f)
    for round in data:
        title = re.sub(r'[\/:*?"<>|]', '', round["source"]["title"])  # Remove invalid filename characters
        
        for speech_id, speech in enumerate(round["speeches"]):
            tmp_segment_list = []
            adus = speech["ADUs"]
            for adu_id, adu in enumerate(adus):
                tmp_segment_dict = {}
                tmp_segment_dict["start"] = 100.0*speech_id+adu_id
                tmp_segment_dict["end"] = 100.0*speech_id+adu_id+1
                tmp_segment_dict["text"] = adu["transcript"]
                tmp_segment_list.append(tmp_segment_dict)
            print(tmp_segment_list)
            tmp_whisper_object = {
                "task": "transcribe",
                "language": "english",
                "duration": 2000.00,
                "segments": tmp_segment_list
            }
            with open (f"dummy_timestamps/{title}_{speech_id}.json", "w") as out_f:
                json.dump(tmp_whisper_object, out_f)
        break

[{'start': 0.0, 'end': 1.0, 'text': "Thank you, Mr. Speaker and ladies and gentlemen in this house. Our side believes that we want to protect most moral people, every people, so every innocent people. And now another thing that's called the technology of hacking system or security camera is developing now and police can capture surprise information in the group of terrorists. So, thanks to those things, we can search those terrorists' move and act. "}, {'start': 1.0, 'end': 2.0, 'text': "And the payment of ransom is money source of terrorist and criminal group. Huge rate of money source is ransom. Plus, criminal is very malicious. So, in many cases, even if victim's side pays huge money to the terrorist group and criminal groups, the victim will be killed by them. This is a very major case, but the family wants to protect their own family or own child, so they give money."}, {'start': 2.0, 'end': 3.0, 'text': "Okay, let's move on to my argument. I have two arguments in this round. Firs

In [30]:
# 6~8スピーチを1つのjsonに統合
import json
import re

output = {}

with open("../corpus/debate_scripts.json") as f:
    data = json.load(f)
    for round in data:
        title = re.sub(r'[\/:*?"<>|]', '', round["source"]["title"])  # Remove invalid filename characters
        
        output = {}
        i = 0
        speech_num = len(round["speeches"])
        for speech_id, speech in enumerate(round["speeches"]):
            side = "prop" if speech_id % 2 == 0 else "opp" #原則
            if speech_id+1 == speech_num-1: #例外
                side="opp"
            elif speech_id+1 == speech_num:
                side="prop"
            
            tmp_segment_list = []
            adus = speech["ADUs"]
            for adu_id, adu in enumerate(adus):
                tmp_segment_dict = {}
                tmp_segment_dict[i] = adu["transcript"]
                tmp_segment_list.append(tmp_segment_dict)
                i+=1
            output[f"{side}{int(speech_id/2)+1}"] = tmp_segment_list
            # with open (f"dummy_timestamps/{title}.json", "w") as out_f:
            #     json.dump(tmp_whisper_object, out_f)
        break

    print(output)

{'prop1': [{0: "Thank you, Mr. Speaker and ladies and gentlemen in this house. Our side believes that we want to protect most moral people, every people, so every innocent people. And now another thing that's called the technology of hacking system or security camera is developing now and police can capture surprise information in the group of terrorists. So, thanks to those things, we can search those terrorists' move and act. "}, {1: "And the payment of ransom is money source of terrorist and criminal group. Huge rate of money source is ransom. Plus, criminal is very malicious. So, in many cases, even if victim's side pays huge money to the terrorist group and criminal groups, the victim will be killed by them. This is a very major case, but the family wants to protect their own family or own child, so they give money."}, {2: "Okay, let's move on to my argument. I have two arguments in this round. Firstly, I'd like to talk about long-term benefits, capturing those terrorists' source 

In [32]:
#load
from dotenv import load_dotenv
from openai import AsyncOpenAI
import os

load_dotenv()
client = AsyncOpenAI()

In [34]:
user_prompt = f"please tell me about the following list: {output}"

response = await client.chat.completions.create(
    model="gpt-4o",
    messages=[
        {"role": "system", "content": "You are a judge in the competitive debate and have to objectively analyze structures of arguments"},
        {"role": "user", "content": user_prompt}
    ]
)

print(response.choices[0].message.content)

This is a detailed record of a debate on the motion of criminalizing the payment of ransom to terrorists. The discussion involves two sides — the proposition (prop) and the opposition (opp).

### Proposition (Prop)

#### Main Arguments:
1. **Moral and Ethical Justifications**:
   - Prop1 (0, 1): Protection of innocent people. Payment of ransom finances terrorist activities. Even when ransoms are paid, victims often get killed.
   - Prop1 (7, 10): Criminalizing ransom payments forces decisions to consult specialists/police which can reduce irrational decisions made under emotional duress.

2. **Long-term Benefits**:
   - Prop1 (2, 4, 6, 8): Preventing terrorists from acquiring financial resources will reduce their capacity to cause harm in the long-term.
   - Prop1 (9): Over the long-term, capturing the source of terrorist funding will save more lives.
   - Prop3 (61, 62): Preventing money flow to terrorists helps in reducing large scale severe terrorist activities.

3. **Comparison and

In [35]:
user_prompt = f"please return the list of tuples of rebuttals i.e., (id of rebuttal source, id of rebuttal target) about the following debate: {output}"

response = await client.chat.completions.create(
    model="gpt-4o",
    messages=[
        {"role": "system", "content": "You are a judge in the competitive debate and have to objectively analyze structures of arguments"},
        {"role": "user", "content": user_prompt}
    ]
)

print(response.choices[0].message.content)

Here is the list of tuples of rebuttals in the given debate transcript:

- (27, 13)
- (34, 27)
- (36, 8)
- (37, 7)
- (38, 13)
- (39, 6)
- (45, 7)
- (60, 27)
- (63, 1)
- (64, 34)
- (65, 23)

These tuples represent cases where one speaker is directly addressing and countering a point made by another specific speaker.


In [37]:
user_prompt = f"please return the list of tuples of rebuttals i.e., (id of rebuttal source, id of rebuttal target) about the following debate: {output}"

response = await client.chat.completions.create(
    model="gpt-4o",
    messages=[
        {"role": "system", "content": "You are a judge in the competitive debate and have to objectively analyze structures of arguments"},
        {"role": "user", "content": user_prompt},
        {"role": "system", "content": "Here is the list of tuples of rebuttals in the given debate transcript: (27, 13), (34, 27), (36, 8), (37, 7), (38, 13), (39, 6), (45, 7), (60, 27), (63, 1), (64, 34), (65, 23). These tuples represent cases where one speaker is directly addressing and countering a point made by another specific speaker."},
        {"role": "user", "content": "随分と少ないな。人間によるアノテーションでは(13, 1), (13, 6), (16, 1), (24, 7), (27, 13), (30, 27), (31, 30), (37, 28), (38, 28), (45, 7), (55, 1), (56, 27), (59, 26), (59, 33), (60, 7), (63, 20), (63, 49), (64, 41), (64, 58), (65, 45), (65, 60), (66, 15), (67, 59)となっていたんだが、どうして違いがあると思う？食い違ったタプル一つ一つに関して、君の解釈と解釈が異なった理由と考えられることを教えてくれ"}
    ]
)

print(response.choices[0].message.content)

このデータの解釈における食い違いを評価するために、一つ一つのタプルに対して、どのようにして違いが生じたのかを見ていきましょう。また、可能な理由を考察します。

1. **(13, 1)**
   - **人間のアノテーション**: これは「terrorist groups の資金源がランサムでない場合がある」(13)が「prop1 でのランサムが唯一の資金源であるという主張」(1)に対しての反論とみなした。
   - **私の解釈**: これは見落としの可能性があります。この反論は明確に見えます。

2. **(13, 6)**
   - **人間のアノテーション**: これは「terrorist groups の資金源がランサムでない場合がある」(13)が「terrorists が武器を購入する資金としてランサムを使う」(6)に対しての反論とみなした。
   - **私の解釈**: 同様に見落としか、反論の関連性を見逃した可能性。

3. **(16, 1)**
   - **人間のアノテーション**: 同じ理由で、「terrorist groups の資金源はランサムに限らない」との反論。
   - **私の解釈**: 同上、見落としの可能性。

4. **(24, 7)**
   - **人間のアノテーション**: 「人々がランサムを支払わなくなると、テロリストは他の方法で資金を得る」などの反論。
   - **私の解釈**: ここも見落としかもしれませんが、7が明確に話されていない可能性。

5. **(27, 13)**
   - **合致**。

6. **(30, 27)**
   - **人間のアノテーション**: 質問として、明らかに27に対して関連。
   - **私の解釈**: 形式的な質問を反論と見做すかどうかの違い。

7. **(31, 30)**
   - **人間のアノテーション**: 質問に対しての回答。
   - **私の解釈**: 形式的な質問回答の認識の違い。

8. **(37, 28)**
   - **人間のアノテーション**: 長期的にはランサムは効果的でない、28の主張に対して。
   - **私の解釈**: 特定の反論として見逃した可能性。

9. **(38, 28)**
   - **人間のアノテーション**:

In [46]:
user_prompt = f"please return the list of tuples of rebuttals i.e., (id of rebuttal source, id of rebuttal target) about the following debate: {output}"

response = await client.chat.completions.create(
    model="gpt-4-turbo",
    messages=[
        {"role": "system", "content": "You are a judge in the competitive debate and have to objectively analyze structures of arguments"},
        {"role": "user", "content": user_prompt}
    ]
)

print(response.choices[0].message.content)

The debate arguments address the issue of ransom payment to terrorists, with the proposition side advocating for criminalizing ransom payments to stop the financing of terrorism, while the opposition focuses on the rights of individuals to protect their loved ones and the potentially negative consequences of criminalizing ransom payments.

Here are the list of tuples of rebuttals, with (id of rebuttal source, id of rebuttal target):

1. (27, 13): Rebuttal from prop2 claiming the opposition understates the significance of the ransom's financial impact on terrorists.
2. (33, 16): Rebuttal from prop2 to opp1’s emphasis on terrorists having multiple sources of income besides ransom, arguing that ransom is still a significant issue.
3. (36, 3): Rebuttal from opp2 questioning the effectiveness and justification of the proposition's plan on police use of technology and surveillance.
4. (37, 8): Rebutal from opp2 to prop1’s claim about long-term benefits of stopping ransom, questioning the mec

In [39]:
print(response.choices[0].message.content)

Based on the given debate, here are the tuples of rebuttals, where (id of rebuttal source, id of rebuttal target):

1. (27, 13)
2. (34, 10)
3. (36, 8)
4. (36, 3)
5. (37, 8)
6. (39, 7)
7. (41, 10)
8. (43, 10)
9. (44, 10)
10. (45, 7)
11. (46, 6)
12. (47, 8)
13. (62, 7)
14. (63, 20)
15. (64, 23)
16. (65, 39)

These tuples indicate the ID of the rebuttal source (the statement making the rebuttal) and the ID of the rebuttal target (the statement being rebutted).


In [41]:
print(response.choices[0].message.content)

```python
rebuttals = [
    (27, 13),
    (34, 0),
    (36, 8),
    (36, 7),
    (39, 7),
    (40, 8),
    (41, 7),
    (42, 7),
    (43, 8),
    (44, 7),
    (49, 7),
    (50, 7),
    (53, 1),
    (57, 0)
]
```


In [43]:
print(response.choices[0].message.content)

Sure, here is the list of tuples of rebuttals, where (id of rebuttal source, id of rebuttal target):

1. (27, 11) - Prop2 refutes Opp1's point about the insignificance of ransom payments.
2. (36, 8) - Opp2 refutes Prop1's point about the effectiveness of police and technology in handling terrorism.
3. (37, 7) - Opp2 refutes Prop1's assertion that criminalizing ransom payments will have long-term benefits in reducing terrorism.
4. (39, 7) - Opp2 contends that criminalizing ransom payments will lead to families feeling betrayed by the state.
5. (45, 7) - Opp2 argues that criminalizing ransom payments will push negotiations underground, making them harder to control.
6. (46, 7) - Opp2 states that secret negotiations by families will lead to higher ransom payments, thus empowering terrorists.
7. (63, 23) - Prop3 refutes Opp2's argument about the inability to save families due to criminalizing ransom payments.
8. (64, 23) - Prop3 argues for the justification of criminalizing ransom payments

In [45]:
print(response.choices[0].message.content) #4turbo

To provide an analysis of the rebuttals within this debate, we need to identify points where speakers directly challenge or refute arguments made by speakers from the opposing sides. Here's a list of identified tuples (rebuttals), which indicate the source speaker who made the rebuttal and the target speaker whose argument was rebutted:

1. (13, 5) - Speaker 13 questions the government side's focus on the funding of terrorism purely through ransom, arguing that terrorists also have other significant sources of income, such as oil fields, which contradicts the government's assumption expressed by speaker 5.
   
2. (14, 7) - Speaker 14 criticizes the government's plan (outlined by speaker 7) for its unrealistic police surveillance expectations and its potential infringement on civil liberties.

3. (16, 6) - Speaker 16 counters speaker 6’s argument about the significant role of ransom in financing terrorism by providing examples of other substantial fundraising methods used by terrorists.

In [47]:
print(response.choices[0].message.content)

The debate arguments address the issue of ransom payment to terrorists, with the proposition side advocating for criminalizing ransom payments to stop the financing of terrorism, while the opposition focuses on the rights of individuals to protect their loved ones and the potentially negative consequences of criminalizing ransom payments.

Here are the list of tuples of rebuttals, with (id of rebuttal source, id of rebuttal target):

1. (27, 13): Rebuttal from prop2 claiming the opposition understates the significance of the ransom's financial impact on terrorists.
2. (33, 16): Rebuttal from prop2 to opp1’s emphasis on terrorists having multiple sources of income besides ransom, arguing that ransom is still a significant issue.
3. (36, 3): Rebuttal from opp2 questioning the effectiveness and justification of the proposition's plan on police use of technology and surveillance.
4. (37, 8): Rebutal from opp2 to prop1’s claim about long-term benefits of stopping ransom, questioning the mec

In [48]:
print(output)

{'prop1': [{0: "Thank you, Mr. Speaker and ladies and gentlemen in this house. Our side believes that we want to protect most moral people, every people, so every innocent people. And now another thing that's called the technology of hacking system or security camera is developing now and police can capture surprise information in the group of terrorists. So, thanks to those things, we can search those terrorists' move and act. "}, {1: "And the payment of ransom is money source of terrorist and criminal group. Huge rate of money source is ransom. Plus, criminal is very malicious. So, in many cases, even if victim's side pays huge money to the terrorist group and criminal groups, the victim will be killed by them. This is a very major case, but the family wants to protect their own family or own child, so they give money."}, {2: "Okay, let's move on to my argument. I have two arguments in this round. Firstly, I'd like to talk about long-term benefits, capturing those terrorists' source 

In [52]:
user_prompt = f"Return the list of tuples of rebuttals i.e., (id of rebuttal source, id of rebuttal target) about the following debate. YOU MUST NOT RETURN ANYTHING OTHER THAN THE LIST OF TUPLES. {output}"

response = await client.chat.completions.create(
    model="gpt-4o",
    messages=[
        {"role": "system", "content": "You are a judge in the competitive debate and have to objectively analyze structures of arguments"},
        {"role": "user", "content": user_prompt}
    ]
)

print(response.choices[0].message.content)

[(27, 13), (36, 8), (37, 2), (38, 9), (39, 9), (45, 14), (46, 14), (63, 34), (64, 34), (65, 39)]


In [53]:
print(response.choices[0].message.content)

[(27, 13), (36, 8), (37, 2), (38, 9), (39, 9), (45, 14), (46, 14), (63, 34), (64, 34), (65, 39)]


In [51]:
print(response.choices[0].message.content)

[(27, 13), (36, 8), (37, 7), (38, 7), (39, 3), (40, 10), (41, 9), (42, 8), (43, 8), (45, 7), (49, 10), (52, 7), (55, 7), (56, 6)]
